De forma sencilla se puede ver que la regla compuesta de Simpson compuesta $S_c(f)$ se escribe como:

$$S_c(f) = \displaystyle \frac{h}{3n} \left [ f(x_0) + f(x_n) + 2 \sum_{i=1}^{\frac{n}{2}-1}f(x_{2i}) + 4 \sum_{i=1}^{\frac{n}{2}}f(x_{2i-1})\right ] $$
con $h=b-a$ y $n$ número de subintervalos (par).

Nota: Los nodos para el caso de Simpson se obtienen con la fórmula: $x_i = a +\frac{i}{2}\hat{h}, \forall i=0,\dots,2n, \hat{h}=\frac{h}{n}$.

Lo siguiente sirve de ayuda. Consideramos un caso de $n=6$ subintervalos, $2n+1=13$ nodos de $x_0$ a $x_{12}$. 

Los siguientes índices de los nodos se multiplican por 4 y son los índices que el subproceso 0 trabajará:

In [1]:
begin=1
end=6

In [2]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


Los siguientes índices de los nodos se multiplican por 2 y son los índices que el subproceso 0 trabajará:

In [3]:
for i in range(begin+1,end+1,2):
    print(i)

2
4
6


In [4]:
begin=7
end=12

Los siguientes índices se multiplican por 4 y son los índices que el subproceso 1 trabajará:

In [5]:
for i in range(begin,end+1,2):
    print(i)

7
9
11


Los siguientes índices se multiplican por 2 y son los índices que el subproceso 1 trabajará:

In [6]:
for i in range(begin+1,end+1,2):
    print(i)

8
10
12


In [7]:
import math
from multiprocessing import Pool 
from scipy.integrate import quad
import time
import multiprocessing as mp

**¿Qué restricciones se deben colocar a las variables $n, ns\text{_}p$ y $p$ para que el programa siguiente obtenga resultados correctos?**

por ejemplo si pruebas con $n=6, p=2, ns\text{_}p=6$ obtienes un error relativo de $10^{-7}$.

**Solución**

Para responder a la pregunta anterior, primero veamos qué pasa cuando $n=5$, $p=2$ y entonces $ns\text{_}p=5$.

Sean las siguientes variables el inicio y fin de los subprocesos 0 y 1, respectivamente:

$begin = (mi\text{_}id*ns\text{_}p)+1 = (0*5)+1=1$

$end = begin-1+ns\text{_}p=1-1+5=5$

$begin = (mi\text{_}id*ns\text{_}p)+1 = (1*5)+1=6$

$end = begin-1+ns\text{_}p=6-1+5=10$

Teniendo así los siguientes índices:

In [8]:
begin=1
end=5

índices que se multiplicarán por 4 en el subproceso 0:

In [9]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


índices que se multiplicarán por 2 en el subproceso 0:

In [10]:
for i in range(begin+1,end+1,2):
    print(i)

2
4


In [11]:
begin=6
end=10

índices que se multiplicarán por 4 en el subproceso 1:

In [12]:
for i in range(begin,end+1,2):
    print(i)

6
8
10


índices que se multiplicarán por 2 en el subproceso 1:

In [13]:
for i in range(begin+1,end+1,2):
    print(i)

7
9


Los índices impares son los que se deben multiplicar por 4 y los pares por 2 y como se puede ver en los for's anteriores en el subproceso 0, sí se está haciendo correctamente pero, en el subproceso 1 los pares se están multiplicando 4 y los impares por 2, lo cuál es incorrecto.

Lo que se debe cumplir para que los índices impares se multipliquen por 4 y los pares por 2 es que la variable begin sea siempre impar. Es decir, esta variable tiene que ser de la forma $2k+1$:

$ begin = (mi\text{_}id*ns\text{_}p)+1 = (2*k)+1$

entonces $mi\text{_}id*ns\text{_}p=2*k$

Lo que quiere decir que el producto anterior debe ser par. Para que un producto entre dos números sea par se deben cumplir los siguientes casos:

1.-$mi\text{_}id$ sea par y $ns\text{_}p$ sea par

2.-$mi\text{_}id$ sea impar y $ns\text{_}p$ sea par

3.-$mi\text{_}id$ sea par y $ns\text{_}p$ sea impar

Sin embargo, el caso 3 no se podría cumplir dado que $mi\text{_}id$ puede ser par o impar, por lo que la única opción de que ésto se cumpla es en los casos 1 y 2, es decir, que $ns\text{_}p$ sea par. Esto significa que: $ns\text{_}p$ es de la forma $(2*m)$:

Entonces:

$$ns\text{_}p=\frac{2*n}{p}=(2*m)$$

Para que el cociente anterior sea par, dado que el numerador es par, $p$ puede ser par o impar y de igual forma $n$ puede ser par o impar, más específicamente:

$\frac{2*n}{p}=2*m$ entonces $\frac{n}{p}=m$ por lo que $n=p*m$

Otro punto importante de observar es que en el algoritmo se le está aplicando la función entero a ns_p, por lo que si el número de nodos $(2*n)$ no es divisible entre el número de subprocesos ($p$), se estarán perdiendo nodos por evalúar.

**Dado lo anterior, $ns\text{_}p$ debe ser par y entero y $n$ tiene que ser múltiplo de $p$ para que el programa siguiente obtenga resultados correctos.**

Ahora haremos la prueba con $n=6, p=2, ns\text{_}p=6$ comprobando que obtenemos una buena aproximación con un error relativo de $10^{-7}$.

In [14]:
a=0
b=1
h=b-a
f=lambda x: math.exp(-x**2)

In [15]:
p=2 #número de subprocesos a lanzar

In [16]:
n=6 #número de subintervalos a lanzar
h_hat=h/n

In [17]:
nt=2*n #variable de apoyo para definir ns_p
ns_p=nt/p

In [18]:
nt

12

In [19]:
ns_p

6.0

In [20]:
ns_p=int(ns_p)
ns_p

6

In [21]:
def Scf_parallel(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    begin = (mi_id*ns_p)+1
    end = begin-1+ns_p
    sum1=0
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    sum1=4*sum1
    sum2=0
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel,range(p))
        aprox=h/(6*n)*(f(a)+sum(results)-f(b)) 
    end_time=time.time()

Para entender un poco mejor el funcionamiento de esta función, a continuación se presenta el siguiente esquema:

<img src="https://dl.dropboxusercontent.com/s/ga8671csua0wzg7/Esquema_Scf_parallel.png?dl=0" heigth="500" width="1500">

**En el bloque del with ¿por qué tenemos que dividir por $\frac{}{6n}$ en lugar de por $\frac{}{3n}$ y por qué tenemos que restar $f(b)$?**


**Solución**

*Primera pregunta*:
Para entenderlo primeramente hay que hacer la observacion de que en la expresión $\frac{h}{3n}$ vista en clase (1.5.Integracion_numerica), considera a n como el total de intervalos (no de subintervalos como aquí). Considerando que para la función Scf_parallel el subintervalo es de tamaño $\frac{b-a}{n}$, el del intervalo es $\frac{b-a}{2n}$ y que $h=b-a$, se tiene:

\begin{equation*}
    \frac{\text{Tamaño del intervalo}}{3} = \frac{\frac{b-a}{2n}}{3} = \frac{b-a}{6n} = \frac{h}{6n}
\end{equation*}

*Segunda pregunta*:
Como se explicó en el esquema de arriba, los subintervalos abarcan $n$ nodos, es decir, valúa $f(x)$ del nodo $1$ al $2n$ (si se cumple que $n$ es múltiplo de $p$), y es justo el último subproceso que valúa la función en el nodo $2n$ y lo multiplica por 2, es decir, agrega $2f(x_{2n})=2f(b)$, algo que no está contemplado en la expresión conocida del algoritmo de integración de simpson, pues en dicha fórmula sólo se suma una vez $f(b)$. Por lo anterior se resta una vez $f(b)$. La siguiente fórmula exhibe esta equivalencia*:

$$ S_c(f) = \displaystyle \frac{h}{6n} \left [ f(x_0) + 2 \left( \sum_{i \text{ par; }0<i\mathbf{\color{red}{<}}2n}f(x_{i}) \right) + 4 \left( \sum_{i \text{ impar; }0<i<2n}f(x_{i}) \right) \mathbf{\color{red}{+}} f(x_{2n}) \right ] =
\displaystyle \frac{h}{6n} \left [ f(x_0) + 2 \left( \sum_{i \text{ par; }0<i \mathbf{\color{red}{\leqslant}} 2n}f(x_{i}) \right) + 4 \left( \sum_{i \text{ impar; }0<i<2n}f(x_{i}) \right) \mathbf{\color{red}{-}} f(x_{2n}) \right ]$$

In [22]:
aprox

0.7468245263791945

In [23]:
obj, err = quad(f, a, b)

In [24]:
obj

0.7468241328124271

In [25]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [26]:
err_relativo(aprox,obj)

5.26987211722925e-07

**¿Qué pasa si $n$ no es par? ¿qué tendríamos que modificar en la implementación si no es par?**

**Solución**

Del análisis hecho anteriormente, concluimos que las restricciones son:

1. $ns\text{_}p$ debe ser entero y par 
2. $n$ debe ser múltiplo de $p$
3. $p$ debe ser mayor o igual a 2 (para aprovechar que está en paralelo)

Además, mencionamos que la restricción de que $n$ fuese par no era un requisito necesario con respecto a la asignación de los índices de los nodos a los $p$ subprocesos. Por ello, hicimos una prueba con $n$ impar cumpliendo las 3 condiciones de arriba:

Veamos qué pasa cuando $n=9$, $p=3$ y entonces $ns\text{_}p=6$.

Sean las siguientes variables el inicio y fin de los subprocesos 0, 1 y 2 respectivamente:

$begin = (mi\text{_}id*ns\text{_}p)+1 = (0*6)+1=1$

$end = begin-1+ns\text{_}p=1-1+6=6$

$begin = (mi\text{_}id*ns\text{_}p)+1 = (1*6)+1=7$

$end = begin-1+ns\text{_}p=7-1+6=12$

$begin = (mi\text{_}id*ns\text{_}p)+1 = (2*6)+1=13$

$end = begin-1+ns\text{_}p=13-1+6=18$

Teniendo así los siguientes índices:

In [27]:
begin=1
end=6

índices que se multiplicarán por 4 en el subproceso 0:

In [28]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


índices que se multiplicarán por 2 en el subproceso 0:

In [29]:
for i in range(begin+1,end+1,2):
    print(i)

2
4
6


In [30]:
begin=7
end=12

índices que se multiplicarán por 4 en el subproceso 1:

In [31]:
for i in range(begin,end+1,2):
    print(i)

7
9
11


índices que se multiplicarán por 2 en el subproceso 1:

In [32]:
for i in range(begin+1,end+1,2):
    print(i)

8
10
12


In [33]:
begin=13
end=18

índices que se multiplicarán por 4 en el subproceso 2:

In [34]:
for i in range(begin,end+1,2):
    print(i)

13
15
17


índices que se multiplicarán por 2 en el subproceso 2:

In [35]:
for i in range(begin+1,end+1,2):
    print(i)

14
16
18


Con esto, verificamos que los índices impares son los que se multiplican por 4 y los pares por 2, es decir, sí se están multiplicando correctamente los sumandos por 2 y por 4 en cada subproceso con la definición de begin y end de cada loop. Por lo que procedemos a correr el programa anterior modificando los parámetros:

In [36]:
a=0
b=1
h=b-a
f=lambda x: math.exp(-x**2)
p=3 #número de subprocesos
n=9 #número de subintervalos a lanzar en este caso es impar, pero debe ser múltiplo de p
h_hat=h/n
nt=2*n #variable de apoyo para definir ns_p
ns_p=nt/p #tiene que ser par

In [37]:
nt

18

In [38]:
ns_p

6.0

In [39]:
ns_p=int(ns_p)
ns_p

6

In [40]:
def Scf_parallel(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    begin = (mi_id*ns_p)+1
    end = begin-1+ns_p
    sum1=0
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    sum1=4*sum1
    sum2=0
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel,range(p))
        aprox=h/(6*n)*(f(a)+sum(results)-f(b)) 
    end_time=time.time()

In [41]:
aprox

0.7468242106299985

In [42]:
obj, err = quad(f, a, b)

In [43]:
obj

0.7468241328124271

In [44]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [45]:
err_relativo(aprox,obj)

1.041979871400868e-07

**Conclusión**

Verificamos que el error relativo se mantiene en 7 dígitos correctos como en el caso de $n=6, p=2, ns\text{_}p=6$. Por ello, podemos concluir que si $n$ no es par el programa sigue arrojando buenas aproximaciones sin tener que hacer modificación alguna pues lo importante es que $n$ sea múltiplo de $p$

tip: haz una prueba con $n=5$ subintervalos y por tanto $2n+1=11$ nodos. Entonces  $\hat{h}=\frac{1}{5}$ y considera extremo izquierdo y derecho a=0, b=1. Luego genera los nodos con la fórmula $x_i=a +\frac{i}{2}\hat{h}, \forall i=0,\dots,2n$ $(2n=10)$ y considera por ejemplo $p=2$ subprocesos. Checa cuáles sumas se les tiene que multiplicar por 2 y por 4. Luego distribuye en partes iguales así como está la implementación anterior en los dos subprocesos. Revisa cómo quedan distribuidos los nodos en los dos subprocesos y cómo inician los for loops **¿se multiplican correctamente los sumandos por 2 y por 4 en cada subproceso con la definición de begin y end de cada loop**

Es un problema de manejo de índices y repartir cantidades iguales a los subprocesos...